In [1]:
%matplotlib inline
import os
os.environ['DATA_PATH'] = "/root/work/data"
import matplotlib.pyplot as plt
from invoke import task
from collections import OrderedDict
import numpy as np
from model import build_dense, build_conv, build_dense_resid, build_ciresan_1, build_ciresan_4
import theano
import theano.tensor as T
from lasagne import layers, objectives, updates
from lasagnekit.datasets.mnist import MNIST
from lasagnekit.datasets.helpers import split
from lasagnekit.datasets.infinite_image_dataset import Transform
from helpers import iterate_minibatches, flip, rotate_scale, rotate_scale_one, elastic_transform, elastic_transform_one
from tabulate import tabulate
from time import time
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from hp_toolkit.search import RandomSearch, DensityFitSearch

from hp_toolkit.hp import Param

In [2]:
class TOM(BaseEstimator):
    def __init__(self, **params):
        params = params.copy()
        self.n_components = params.get('n_components')
        del params['n_components']
        self.clf = Pipeline([
            ('pca', PCA(n_components=self.n_components)),
            ('clf', SVC(random_state=42, probability=True, **params)),
        ])
 
    def fit(self, X, y):
        self.clf.fit(X, y)
 
    def predict(self, X):
        return self.clf.predict(X)
 
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

class CHERTI(BaseEstimator):
    def __init__(self, n_estimators=20, max_leaf_nodes=5):
        self.n_estimators = n_estimators
        self.max_leaf_nodes = max_leaf_nodes
 
    def fit(self, X, y):
        self.clf = AdaBoostClassifier(
            n_estimators=self.n_estimators, 
            base_estimator=DecisionTreeClassifier(max_leaf_nodes=self.max_leaf_nodes)
        )
        return self.clf.fit(X, y)
 
    def predict(self, X):
        return self.clf.predict(X)
 
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [3]:
w, h, c = 28, 28, 1
print('Loading data...')

def preprocess(data):
    data = data * 2 - 1
    return data.reshape((data.shape[0], c, w, h))

train_full = MNIST(which='train')
train_full.load()
#train_full.X = preprocess(train_full.X)
#train_full.X = train_full.X[0:128*10]

train, valid = split(train_full, test_size=0.16667) # 10000 examples in validation set

test = MNIST(which='test')
test.load()
#test.X = preprocess(test.X)

Loading data...


In [ ]:
def epsilon():
    t = 1
    while True:
        eps = 1. / t
        yield eps
        t += 1
        
CLS = CHERTI

if CLS == RandomForestClassifier:
    hp = {
        "max_depth": Param(initial=1, interval=[1, 20], type='int'),
        "n_estimators": Param(initial=10, interval=[10, 300], type='int')
    }
if CLS == CHERTI:
    hp = {
        "max_leaf_nodes": Param(initial=1, interval=[1, 100], type='int'),
        "n_estimators": Param(initial=10, interval=[10, 50], type='int')
    }
if CLS == TOM:
    hp = {
        "C": Param(initial=1, interval=[1, 1000], type='real', scale='log10'),
        "n_components": Param(initial=100, interval=[10, 300], type='int'),
        "kernel": Param(initial='rbf', interval=['rbf', 'poly', 'linear'], type='choice')
    }


#search = RandomSearch(hp)
search = DensityFitSearch(hp, epsilon=epsilon, keep=10)

smallest_err = np.inf

for i in range(50):
    params = search.sample_next()
    clf = CLS(**params)
    clf.fit(train.X, train.y)
    err = (clf.predict(valid.X) != valid.y).mean()
    search.update(params, err)
    if err < smallest_err:
        best_params = params
        smallest_err = err
        print("best params : {},  error :{}".format(params, err))
        best_model = clf

best params : {'n_estimators': 16, 'max_leaf_nodes': 87},  error :0.1401
best params : {'n_estimators': 40, 'max_leaf_nodes': 30},  error :0.1286
best params : {'n_estimators': 24, 'max_leaf_nodes': 83},  error :0.1188
best params : {'n_estimators': 45, 'max_leaf_nodes': 45},  error :0.1143
best params : {'n_estimators': 41, 'max_leaf_nodes': 90},  error :0.0946


In [7]:
err = (best_model.predict(test.X) != test.y).mean()
print(err)

0.0939
